In [ ]:
###############################################################################
# The Institute for the Design of Advanced Energy Systems Integrated Platform
# Framework (IDAES IP) was produced under the DOE Institute for the
# Design of Advanced Energy Systems (IDAES), and is copyright (c) 2018-2022
# by the software owners: The Regents of the University of California, through
# Lawrence Berkeley National Laboratory,  National Technology & Engineering
# Solutions of Sandia, LLC, Carnegie Mellon University, West Virginia University
# Research Corporation, et al.  All rights reserved.
#
# Please see the files COPYRIGHT.md and LICENSE.md for full copyright and
# license information.
###############################################################################

# Perovskite Design

This notebook serves as an example application of the MatOpt framework on bulk nanostructured materials. 
We consider the problem of how to optimally place dopant in a perovskite lattice. 

For more information, see:
Hanselman, Christopher L., et al. "A framework for optimizing oxygen vacancy formation in doped perovskites." *Computers & Chemical Engineering* 126 (2019): 168-177. DOI: [10.1016/j.compchemeng.2019.03.033](https://doi.org/10.1016/j.compchemeng.2019.03.033)

## Importing Packages

We start by importing MatOpt.

In [ ]:
import numpy as np
from idaes.apps.matopt import *

## Representing Materials

First, we construct a ***Lattice*** object to hold information about the sites in our material. For this application, we use the ***PerovskiteLattice*** class with lattice constants *A*, *B*, and *C*. 

In [ ]:
A = 4.0
B = 4.0
C = 4.0
Lat = PerovskiteLattice(A, B, C)

Next, we construct ***Shape*** and ***Tiling*** objects to help define the material locations of interest. In this case, we use ***RectPrism*** and ***CubicTiling***, respectively. 

Note that we shift the shape slightly to avoid ambiguity about which sites on the border of the cell should be included. 

In [ ]:
nUnitCellsOnEdge = 2
S = RectPrism(nUnitCellsOnEdge * A, nUnitCellsOnEdge * B, nUnitCellsOnEdge * C)
S.shift(np.array([-0.01, -0.01, -0.01]))
T = CubicTiling(S)

Next, we construct the ***Canvas*** object which will hold all the information about the sites and information about neighbors. We also define a list of ***Atom*** objects that serve as the building blocks of our material.

In [ ]:
Canv = Canvas.fromLatticeAndTilingScan(Lat, T)
Atoms = [Atom("Ba"), Atom("Fe"), Atom("In"), Atom("O")]

Finally, we load a list of conformations from file that represent a set of dopant configurations that we would like to indicate in the design. 

In [ ]:
from tempfile import TemporaryDirectory
from zipfile import ZipFile

iDesiredConfs = [
    394,
    395,
    396,
    397,
    398,
    399,
    400,
    401,
    68,
    69,
    70,
    71,
    162,
    163,
    164,
    165,
    166,
    167,
    168,
    169,
]
with TemporaryDirectory() as ConfDir:
    ZipFile("./confs.zip").extractall(ConfDir)
    ConfDesigns = loadFromPDBs(
        [str(i) + ".pdb" for i in iDesiredConfs], folder=ConfDir + "/confs/"
    )
Confs = [Conf.Contents for Conf in ConfDesigns]

## Building the Model

To begin specifying the model, we first define several pieces of information that will help specify the design problem.

In [ ]:
Sites = [i for i in range(len(Canv))]
ASites = [i for i in Sites if Lat.isASite(Canv.Points[i])]
BSites = [i for i in Sites if Lat.isBSite(Canv.Points[i])]
OSites = [i for i in Sites if Lat.isOSite(Canv.Points[i])]
pctLocalLB, pctLocalUB = 0, 1
pctGlobalLB, pctGlobalUB = 0.0, 0.3
LocalBounds = {
    (i, Atom("In")): (
        round(pctLocalLB * len(Canv.NeighborhoodIndexes[i])),
        round(pctLocalUB * len(Canv.NeighborhoodIndexes[i])),
    )
    for i in OSites
}
GlobalLB = round(pctGlobalLB * len(BSites))
GlobalUB = round(pctGlobalUB * len(BSites))

Next, we initialize a ***MatOptModel*** object that will hold all the information about material descriptors and desired functionalities. 

In [ ]:
m = MatOptModel(Canv, Atoms, Confs)

By default, several basic variables are pre-defined. See the first example, ***Monometallic_Nanocluster_Design.ipynb*** for a description of basic variables, expressions, and constraint rules.

For this system, we introduce several rules about the allowed placement of atoms in the design. First, we require that all A-sites in the material are occupied by Ba. Next, we require that all O-sites are occupied by O. Thirdly, we forbid Ba and O from being placed in B-sites. And finally, we require that some atom be placed in each B-site. These four rules  effectively limit the scope of the optimization to focus on the labeling of B-sites as either Fe or In. 

In [ ]:
m.Yik.rules.append(FixedTo(1, sites=ASites, site_types=[Atom("Ba")]))
m.Yik.rules.append(FixedTo(1, sites=OSites, site_types=[Atom("O")]))
m.Yik.rules.append(FixedTo(0, sites=BSites, site_types=[Atom("Ba"), Atom("O")]))
m.Yi.rules.append(FixedTo(1, sites=BSites))

To specify additional constraints to the model, we create several descriptors for the activity, local dopant concentration, and the global dopant concentration. 

Notice that in each case, we specify a subset of locations or atoms of interest. This is because, for example, our material activity depends on oxygen sites only and it would be nonsensical to try to interpret one of the conformations on a different type of site. Similarly, the dopant budgets are written only over In atoms and not on Ba, Fe, or O. 

In [ ]:
m.addGlobalDescriptor(
    "Activity",
    rules=EqualTo(SumSitesAndConfs(m.Zic, coefs=1 / len(OSites), sites_to_sum=OSites)),
)
m.addGlobalTypesDescriptor(
    "GlobalIndiumConc",
    bounds=(GlobalLB, GlobalUB),
    rules=EqualTo(SumSites(m.Yik, site_types=[Atom("In")], sites_to_sum=BSites)),
)
m.addSitesTypesDescriptor(
    "LocalIndiumConc",
    bounds=LocalBounds,
    rules=EqualTo(SumNeighborSites(m.Yik, sites=OSites, site_types=[Atom("In")])),
)

## Solving the Model

Given a fully formed model, we can optimize by maximizing or minimizing one of the global descriptors.

In [ ]:
D = None
try:
    D = m.maximize(m.Activity, tilim=360)
except:
    print("MaOpt can not find usable solver (CPLEX or NEOS-CPLEX)")

## Processing Solutions

If the optimizer was successful in finding an optimal (or just feasible) solution, we can plot the resulting design to any of several standard file formats. However, it is often useful to modify the design to highlight key features. Here, we label all O-sites that constitute one of the desired conformations by replacing the atom with an S.

In [ ]:
if D is not None:
    for i, c in m.Zic.keys():
        if m.Zic.values[i, c] > 0.5:
            D.setContent(i, Atom("S"))
    D.toCFG("result.cfg", BBox=S)